In [1]:
import pandas as pd
import numpy as np
import statistics as stats

import gurobipy as gp
from gurobipy import GRB

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)

In [2]:
dfs = []

## 2020

### Preprocessing

In [3]:
### Load Datasets

# Stats
StatsQB2020 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/StatsQB2020.csv")
StatsRB2020 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/StatsRB2020.csv")
StatsWR2020_Part1 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/StatsWR2020_Part1.csv")
StatsWR2020_Part2 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/StatsWR2020_Part2.csv")
StatsTE2020 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/StatsTE2020.csv")
StatsDST2020 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/StatsDST2020.csv")

# Salaries
SalariesQB2020 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/SalariesQB2020.csv")
SalariesRB2020_Part1 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/SalariesRB2020_Part1.csv")
SalariesRB2020_Part2 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/SalariesRB2020_Part2.csv")
SalariesRB2020_Part3 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/SalariesRB2020_Part3.csv")
SalariesWR2020_Part1 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/SalariesWR2020_Part1.csv")
SalariesWR2020_Part2 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/SalariesWR2020_Part2.csv")
SalariesWR2020_Part3 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/SalariesWR2020_Part3.csv")
SalariesTE2020_Part1 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/SalariesTE2020_Part1.csv")
SalariesTE2020_Part2 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/SalariesTE2020_Part2.csv")
SalariesTE2020_Part3 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/SalariesTE2020_Part3.csv")
SalariesDST2020 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2020/SalariesDST2020.csv")

### Combine Datasets that have parts
StatsWR2020 = StatsWR2020_Part1.append(StatsWR2020_Part2)
SalariesRB2020 = SalariesRB2020_Part1.append(SalariesRB2020_Part2).append(SalariesRB2020_Part3)
SalariesWR2020 = SalariesWR2020_Part1.append(SalariesWR2020_Part2).append(SalariesWR2020_Part3)
SalariesTE2020 = SalariesTE2020_Part1.append(SalariesTE2020_Part2).append(SalariesTE2020_Part3)

### Merge DataFrames (RB, WR, FLEX)
selected_columns = ['PlayerID','Name_x','Position_x','Week','Team_x','Opponent_x','DraftKingsSalary','FantasyPointsDraftKings_y','FantasyPointsDraftKings_x']
renamed_columns = ['PlayerID','Name','Position','Week','Team','Opponent','Salary','FantasyPoints','FantasyPoints_DraftKingsProjection']

QB2020 = pd.merge(SalariesQB2020, StatsQB2020, how="inner", left_on=["PlayerID", "Week"], right_on=["PlayerID", "Week"])
QB2020 = QB2020[selected_columns]
QB2020.columns = renamed_columns

RB2020 = pd.merge(SalariesRB2020, StatsRB2020, how="inner", left_on=["PlayerID", "Week"], right_on=["PlayerID", "Week"])
RB2020 = RB2020[selected_columns]
RB2020.columns = renamed_columns

WR2020 = pd.merge(SalariesWR2020, StatsWR2020, how="inner", left_on=["PlayerID", "Week"], right_on=["PlayerID", "Week"])
WR2020 = WR2020[selected_columns]
WR2020.columns = renamed_columns

TE2020 = pd.merge(SalariesTE2020, StatsTE2020, how="inner", left_on=["PlayerID", "Week"], right_on=["PlayerID", "Week"])
TE2020 = TE2020[selected_columns]
TE2020.columns = renamed_columns

DST2020 = pd.merge(SalariesDST2020, StatsDST2020, how="inner", left_on=["PlayerID", "Week"], right_on=["PlayerID", "Week"])
DST2020 = DST2020[selected_columns]
DST2020.columns = renamed_columns


In [4]:
QB2020['QB'] = [1] * len(QB2020)
RB2020['RB'] = [1] * len(RB2020)
WR2020['WR'] = [1] * len(WR2020)
TE2020['TE'] = [1] * len(TE2020)
DST2020['DST'] = [1] * len(DST2020)

In [5]:
### Add Fantasy Points rank column
def FPrank(df):
    frames = []
    for i in range(1,18):
        frame = df[df.Week == i].sort_values(by='FantasyPoints', ascending=False)
        frame['FPRank'] = frame['FantasyPoints'].rank(ascending=False, method='min')
        frames.append(frame)
    final = pd.concat(frames)
    return final

QB2020 = FPrank(QB2020)
RB2020 = FPrank(RB2020)
WR2020 = FPrank(WR2020)
TE2020 = FPrank(TE2020)
DST2020 = FPrank(DST2020)

In [6]:
### Add Salary rank column
def Srank(df):
    frames = []
    for i in range(1,18):
        frame = df[df.Week == i].sort_values(by='Salary', ascending=False)
        frame['SRank'] = frame['Salary'].rank(ascending=False, method='min')
        frames.append(frame)
    final = pd.concat(frames)
    return final

QB2020 = Srank(QB2020)
RB2020 = Srank(RB2020)
WR2020 = Srank(WR2020)
TE2020 = Srank(TE2020)
DST2020 = Srank(DST2020)

In [7]:
### Combine all players into 1 dataset
players = (QB2020, RB2020, WR2020, TE2020, DST2020)
all_players = pd.concat(players, ignore_index = True).fillna(0)

###  Perfect Hindsight Model

In [8]:
### Function: Optimization model that selects best fantasy football lineup
def optimize_week(weeknum):

    m = gp.Model()

    # add binary variables for every player in week i
    players_temp = all_players[all_players.Week ==weeknum]
    keys = [*range(0, len(players_temp))]
    players_temp.index = keys

    # define variables
    player = m.addVars(len(players_temp), vtype=GRB.BINARY, name='player')
    cost = players_temp.Salary
    points = players_temp.FantasyPoints 
    qb = players_temp.QB
    rb = players_temp.RB
    wr = players_temp.WR
    te = players_temp.TE
    dst = players_temp.DST

    ### Objective Function: Maximize Fantasy League Points
    length = len(players_temp)
    m.setObjective((gp.quicksum(points[x] * player[x] for x in range(length))), GRB.MAXIMIZE)

    ### Constraint 1: Positions
    # 1 QB
    m.addConstr(1 == gp.quicksum(player[x]*qb[x] for x in range(length)))

    # 2 RB (and FB) & maybe flex
    m.addConstr(2 <= sum(player[x]*rb[x] for x in range(length)))
    m.addConstr(3 >= sum(player[x]*rb[x] for x in range(length)))

    # 3 WR & maybe flex
    m.addConstr(3 <= gp.quicksum(player[x]*wr[x] for x in range(length)))
    m.addConstr(4 >= gp.quicksum(player[x]*wr[x] for x in range(length)))

    # 1 TE & maybe flex
    m.addConstr(1 <= gp.quicksum(player[x]*te[x] for x in range(length)))
    m.addConstr(2 >= gp.quicksum(player[x]*te[x] for x in range(length)))

    # 1 DST (Defense)
    m.addConstr(1 == gp.quicksum(player[x]*dst[x] for x in range(length)))

    # 1 FLEX: add an extra RB, WR or TE (total 9 players)
    m.addConstr(9 == gp.quicksum(player[x]*qb[x] + player[x]*rb[x] + player[x]*wr[x] + 
    player[x]*te[x] + player[x]*dst[x] for x in range(length)))


    ### Constraint 2: Budget
    # Total team budget for each week cannot exceed $50,000
    m.addConstr(50000 >= gp.quicksum(player[x]*cost[x] for x in range(length)))

    status = m.optimize()

    # Output and analysis
    ObjectiveValue=m.objVal

    #The optimal solution consists of the following
    dict_week_i = []
    for v in m.getVars():
        if v.x == 1: 
            # extract number (aka key) from variable name
            one = v.VarName.index('[')
            two = v.VarName.index(']')
            temp = int(v.VarName[one+1:two])

            row_player = players_temp.loc[[0]]
           
            dict_player_v = {"FPRank": players_temp.FPRank[temp],
                             "SRank": players_temp.SRank[temp],
                             "Name": players_temp.Name[temp],
                             "Position": players_temp.Position[temp],
                             "Week": players_temp.Week[temp],
                             "Team": players_temp.Team[temp],
                             "Opponent": players_temp.Opponent[temp],
                             "Salary": players_temp.Salary[temp],
                             "FantasyPoints": players_temp.FantasyPoints[temp]}

            dict_week_i.append(dict_player_v)

    df_week_i = pd.DataFrame(dict_week_i)
    
    position_categories = ["QB", "RB", "WR", "TE", "FLEX", "DST"] 
    df_week_i["Position"] = pd.Categorical(df_week_i["Position"], categories = position_categories)
    df_week_i.sort_values(by = "Position")
    
    return df_week_i

In [9]:
### Run the optimization model on weeks 1-16
for i in range(1,17):
    df = optimize_week(i)
    dfs.append(df)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-01-24
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10 rows, 456 columns and 1754 nonzeros
Model fingerprint: 0x2bf72841
Variable types: 0 continuous, 456 integer (456 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e-01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+04]
Found heuristic solution: objective 64.9000000
Presolve removed 1 rows and 184 columns
Presolve time: 0.00s
Presolved: 9 rows, 272 columns, 1070 nonzeros
Found heuristic solution: objective 288.7800000
Variable types: 0 continuous, 272 integer (264 binary)

Root relaxation: objective 2.891657e+02, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumben

  Matrix range     [1e+00, 8e+03]
  Objective range  [4e-02, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+04]
Found heuristic solution: objective 100.2800000
Presolve removed 0 rows and 171 columns
Presolve time: 0.00s
Presolved: 10 rows, 224 columns, 869 nonzeros
Found heuristic solution: objective 323.4200000
Variable types: 0 continuous, 224 integer (211 binary)

Root relaxation: cutoff, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0       323.42000  323.42000  0.00%     -    0s

Explored 1 nodes (5 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 2: 323.42 100.28 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.234200000000e+02, best bound 3.234200000000e+02, gap 0.0000%
Gurobi Optimizer version 9.5.

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10 rows, 422 columns and 1610 nonzeros
Model fingerprint: 0xab16d4d2
Variable types: 0 continuous, 422 integer (422 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e-01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+04]
Found heuristic solution: objective 61.0400000
Presolve removed 0 rows and 175 columns
Presolve time: 0.00s
Presolved: 10 rows, 247 columns, 964 nonzeros
Found heuristic solution: objective 300.6600000
Variable types: 0 continuous, 247 integer (237 binary)

Root relaxation: objective 3.025556e+02, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  302.55563    0    2  300.66000  302.55563  

In [10]:
### Ouputs selected lineups for each week
for i in range(0,len(dfs)):
    display(dfs[i])

,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,3.0,Russell Wilson,QB,1,SEA,ATL,7000.0,34.78
1,1.0,10.0,Josh Jacobs,RB,1,LV,CAR,6800.0,35.90
2,4.0,63.0,Nyheim Hines,RB,1,IND,JAX,4000.0,27.30
3,1.0,4.0,Davante Adams,WR,1,GB,MIN,7300.0,44.60
4,3.0,9.0,Adam Thielen,WR,1,MIN,GB,6700.0,34.00
5,2.0,21.0,Calvin Ridley,WR,1,ATL,SEA,6100.0,36.90
6,5.0,49.0,Darius Slayton,WR,1,NYG,PIT,4700.0,31.20
7,1.0,19.0,Dallas Goedert,TE,1,PHI,WAS,4100.0,27.10
8,1.0,20.0,New Orleans Saints,DST,1,NO,TB,2400.0,17.00


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,4.0,Dak Prescott,QB,2,DAL,ATL,6800.0,43.8
1,2.0,5.0,Alvin Kamara,RB,2,NO,LV,7800.0,38.4
2,1.0,9.0,Aaron Jones,RB,2,GB,DET,7100.0,48.6
3,1.0,6.0,Calvin Ridley,WR,2,ATL,DAL,6800.0,32.9
4,2.0,8.0,Stefon Diggs,WR,2,BUF,MIA,6500.0,32.3
5,4.0,24.0,Terry McLaurin,WR,2,WAS,ARI,5900.0,28.5
6,2.0,17.0,Mike Gesicki,TE,2,MIA,BUF,4000.0,30.0
7,5.0,53.0,Jordan Reed,TE,2,SF,NYJ,2600.0,24.0
8,1.0,22.0,Indianapolis Colts,DST,2,IND,MIN,2500.0,15.0


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,2.0,Patrick Mahomes,QB,3,KC,BAL,7400.0,43.0
1,1.0,2.0,Alvin Kamara,RB,3,NO,GB,7900.0,47.7
2,2.0,56.0,Rex Burkhead,RB,3,NE,LV,4000.0,34.8
3,1.0,11.0,Tyler Lockett,WR,3,SEA,DAL,6400.0,40.0
4,4.0,14.0,Allen Robinson II,WR,3,CHI,ATL,6200.0,31.3
5,2.0,17.0,Keenan Allen,WR,3,LAC,CAR,6100.0,34.2
6,3.0,56.0,Justin Jefferson,WR,3,MIN,TEN,4200.0,33.5
7,1.0,27.0,Jimmy Graham,TE,3,CHI,ATL,3400.0,24.0
8,1.0,1.0,Indianapolis Colts,DST,3,IND,NYJ,4100.0,26.0


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,4.0,Dak Prescott,QB,4,DAL,CLE,7200.0,42.28
1,2.0,3.0,Dalvin Cook,RB,4,MIN,HOU,7600.0,31.60
2,1.0,16.0,Joe Mixon,RB,4,CIN,JAX,5800.0,45.10
3,2.0,7.0,Amari Cooper,WR,4,DAL,CLE,6700.0,36.40
4,3.0,17.0,DJ Chark Jr.,WR,4,JAX,CIN,6000.0,29.50
5,1.0,20.0,Odell Beckham Jr.,WR,4,CLE,DAL,5800.0,38.40
6,1.0,1.0,George Kittle,TE,4,SF,PHI,6300.0,43.10
7,2.0,24.0,Robert Tonyan,TE,4,GB,ATL,3400.0,33.80
8,1.0,0.0,Kansas City Chiefs,DST,4,KC,NE,0.0,20.00


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,3.0,Patrick Mahomes,QB,5,KC,LV,7700.0,33.7
1,1.0,10.0,Mike Davis,RB,5,CAR,ATL,6400.0,29.9
2,2.0,18.0,Todd Gurley II,RB,5,ATL,CAR,5700.0,28.0
3,1.0,58.0,Chase Claypool,WR,5,PIT,PHI,4100.0,45.6
4,3.0,58.0,Brandin Cooks,WR,5,HOU,JAX,4100.0,33.1
5,4.0,65.0,Mike Williams,WR,5,LAC,NO,3700.0,30.9
6,2.0,81.0,Travis Fulgham,WR,5,PHI,PIT,3000.0,34.2
7,1.0,2.0,Travis Kelce,TE,5,KC,LV,6400.0,27.8
8,1.0,4.0,Baltimore Ravens,DST,5,BAL,CIN,3900.0,26.0


...

,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,4.0,Deshaun Watson,QB,12,HOU,DET,7400.0,36.12
1,1.0,3.0,Derrick Henry,RB,12,TEN,IND,7900.0,41.50
2,2.0,14.0,Antonio Gibson,RB,12,WAS,DAL,6000.0,39.60
3,1.0,4.0,Tyreek Hill,WR,12,KC,TB,7800.0,60.90
4,2.0,12.0,William Fuller V,WR,12,HOU,DET,6400.0,38.10
5,3.0,35.0,Jarvis Landry,WR,12,CLE,JAX,5200.0,31.30
6,12.0,91.0,Collin Johnson,WR,12,JAX,CLE,3000.0,19.60
7,4.0,21.0,Robert Tonyan,TE,12,GB,CHI,3400.0,17.70
8,1.0,28.0,Atlanta Falcons,DST,12,ATL,LV,2200.0,28.00


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,15.0,Derek Carr,QB,13,LV,NYJ,5800.0,35.74
1,1.0,22.0,David Montgomery,RB,13,CHI,DET,5500.0,27.10
2,5.0,54.0,Ty Johnson,RB,13,NYJ,LV,4000.0,22.70
3,2.0,2.0,Davante Adams,WR,13,GB,PHI,9000.0,37.10
4,4.0,12.0,Justin Jefferson,WR,13,MIN,JAX,6900.0,30.30
5,1.0,34.0,Corey Davis,WR,13,TEN,CLE,5100.0,38.20
6,3.0,38.0,Cole Beasley,WR,13,BUF,SF,4900.0,31.00
7,1.0,2.0,Darren Waller,TE,13,LV,NYJ,6100.0,48.00
8,1.0,22.0,New England Patriots,DST,13,NE,LAC,2400.0,31.00


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,7.0,Lamar Jackson,QB,14,BAL,CLE,7000.0,37.92
1,1.0,2.0,Derrick Henry,RB,14,TEN,JAX,8700.0,39.20
2,3.0,15.0,Miles Sanders,RB,14,PHI,NO,6200.0,32.60
3,2.0,20.0,Jonathan Taylor,RB,14,IND,LV,5800.0,33.50
4,2.0,13.0,Allen Robinson II,WR,14,CHI,HOU,6800.0,30.30
5,7.0,36.0,T.Y. Hilton,WR,14,IND,LV,5100.0,25.60
6,11.0,93.0,KJ Hamler,WR,14,DEN,CAR,3000.0,22.60
7,2.0,7.0,Mike Gesicki,TE,14,MIA,KC,4500.0,23.50
8,1.0,14.0,Washington Commanders,DST,14,WAS,SF,2800.0,23.00


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,15.0,Jalen Hurts,QB,15,PHI,ARI,5900.0,40.82
1,1.0,9.0,David Montgomery,RB,15,CHI,MIN,7000.0,32.20
2,2.0,24.0,Tony Pollard,RB,15,DAL,SF,5200.0,31.20
3,6.0,26.0,David Johnson,RB,15,HOU,IND,5100.0,27.30
4,1.0,4.0,Calvin Ridley,WR,15,ATL,TB,8200.0,35.30
5,3.0,27.0,Marvin Jones Jr.,WR,15,DET,TEN,5700.0,30.20
6,5.0,88.0,Zach Pascal,WR,15,IND,HOU,3200.0,24.90
7,1.0,2.0,Darren Waller,TE,15,LV,LAC,6900.0,33.00
8,1.0,20.0,Dallas Cowboys,DST,15,DAL,SF,2700.0,15.00


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,7.0,Josh Allen,QB,16,BUF,NE,7300.0,35.3
1,2.0,25.0,Myles Gaskin,RB,16,MIA,LV,5300.0,33.9
2,1.0,0.0,Alvin Kamara,RB,16,NO,MIN,0.0,59.2
3,1.0,1.0,Davante Adams,WR,16,GB,TEN,9100.0,46.2
4,2.0,5.0,Stefon Diggs,WR,16,BUF,NE,8000.0,44.5
5,3.0,15.0,Mike Evans,WR,16,TB,DET,6100.0,43.1
6,4.0,46.0,Michael Gallup,WR,16,DAL,PHI,4100.0,33.1
7,1.0,0.0,Irv Smith Jr.,TE,16,MIN,NO,0.0,23.3
8,1.0,18.0,Carolina Panthers,DST,16,CAR,WAS,2700.0,21.0


## 2019

### Preprocessing

In [11]:
### Load Datasets

# Stats
StatsQB2019 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/StatsQB2019.csv")
StatsRB2019 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/StatsRB2019.csv")
StatsWR2019_Part1 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/StatsWR2019_Part1.csv")
StatsWR2019_Part2 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/StatsWR2019_Part2.csv")
StatsTE2019 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/StatsTE2019.csv")
StatsDST2019 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/StatsDST2019.csv")

# Salaries
SalariesQB2019 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/SalariesQB2019.csv")
SalariesRB2019_Part1 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/SalariesRB2019_Part1.csv")
SalariesRB2019_Part2 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/SalariesRB2019_Part2.csv")
SalariesRB2019_Part3 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/SalariesRB2019_Part3.csv")
SalariesWR2019_Part1 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/SalariesWR2019_Part1.csv")
SalariesWR2019_Part2 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/SalariesWR2019_Part2.csv")
SalariesWR2019_Part3 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/SalariesWR2019_Part3.csv")
SalariesTE2019_Part1 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/SalariesTE2019_Part1.csv")
SalariesTE2019_Part2 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/SalariesTE2019_Part2.csv")
SalariesTE2019_Part3 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/SalariesTE2019_Part3.csv")
SalariesDST2019 = pd.read_csv("~/Desktop/MGSC 434 Final Project/Raw Data 2019/SalariesDST2019.csv")

### Combine Datasets that have parts
StatsWR2019 = StatsWR2019_Part1.append(StatsWR2019_Part2)
SalariesRB2019 = SalariesRB2019_Part1.append(SalariesRB2019_Part2).append(SalariesRB2019_Part3)
SalariesWR2019 = SalariesWR2019_Part1.append(SalariesWR2019_Part2).append(SalariesWR2019_Part3)
SalariesTE2019 = SalariesTE2019_Part1.append(SalariesTE2019_Part2).append(SalariesTE2019_Part3)

### Merge DataFrames (RB, WR, FLEX)
selected_columns = ['PlayerID','Name_x','Position_x','Week','Team_x','Opponent_x','DraftKingsSalary','FantasyPointsDraftKings_y','FantasyPointsDraftKings_x']
renamed_columns = ['PlayerID','Name','Position','Week','Team','Opponent','Salary','FantasyPoints','FantasyPoints_DraftKingsProjection']

QB2019 = pd.merge(SalariesQB2019, StatsQB2019, how="inner", left_on=["PlayerID", "Week"], right_on=["PlayerID", "Week"])
QB2019 = QB2019[selected_columns]
QB2019.columns = renamed_columns

RB2019 = pd.merge(SalariesRB2019, StatsRB2019, how="inner", left_on=["PlayerID", "Week"], right_on=["PlayerID", "Week"])
RB2019 = RB2019[selected_columns]
RB2019.columns = renamed_columns

WR2019 = pd.merge(SalariesWR2019, StatsWR2019, how="inner", left_on=["PlayerID", "Week"], right_on=["PlayerID", "Week"])
WR2019 = WR2019[selected_columns]
WR2019.columns = renamed_columns

TE2019 = pd.merge(SalariesTE2019, StatsTE2019, how="inner", left_on=["PlayerID", "Week"], right_on=["PlayerID", "Week"])
TE2019 = TE2019[selected_columns]
TE2019.columns = renamed_columns

DST2019 = pd.merge(SalariesDST2019, StatsDST2019, how="inner", left_on=["PlayerID", "Week"], right_on=["PlayerID", "Week"])
DST2019 = DST2019[selected_columns]
DST2019.columns = renamed_columns


In [12]:
QB2019['QB'] = [1] * len(QB2019)
RB2019['RB'] = [1] * len(RB2019)
WR2019['WR'] = [1] * len(WR2019)
TE2019['TE'] = [1] * len(TE2019)
DST2019['DST'] = [1] * len(DST2019)

In [13]:
### Add Fantasy Points rank column
def FPrank(df):
    frames = []
    for i in range(1,18):
        frame = df[df.Week == i].sort_values(by='FantasyPoints', ascending=False)
        frame['FPRank'] = frame['FantasyPoints'].rank(ascending=False, method='min')
        frames.append(frame)
    final = pd.concat(frames)
    return final

QB2019 = FPrank(QB2019)
RB2019 = FPrank(RB2019)
WR2019 = FPrank(WR2019)
TE2019 = FPrank(TE2019)
DST2019 = FPrank(DST2019)

In [14]:
19### Add Salary rank column
def Srank(df):
    frames = []
    for i in range(1,18):
        frame = df[df.Week == i].sort_values(by='Salary', ascending=False)
        frame['SRank'] = frame['Salary'].rank(ascending=False, method='min')
        frames.append(frame)
    final = pd.concat(frames)
    return final

QB2019 = Srank(QB2019)
RB2019 = Srank(RB2019)
WR2019 = Srank(WR2019)
TE2019 = Srank(TE2019)
DST2019 = Srank(DST2019)

In [15]:
### Combine all players into 1 dataset
players = (QB2019, RB2019, WR2019, TE2019, DST2019)
all_players = pd.concat(players, ignore_index = True).fillna(0)

###  Perfect Hindsight Model

In [16]:
### Function: Optimization model that selects best fantasy football lineup
def optimize_week(weeknum):

    m = gp.Model()

    # add binary variables for every player in week i
    players_temp = all_players[all_players.Week ==weeknum]
    keys = [*range(0, len(players_temp))]
    players_temp.index = keys

    # define variables
    player = m.addVars(len(players_temp), vtype=GRB.BINARY, name='player')
    cost = players_temp.Salary
    points = players_temp.FantasyPoints 
    qb = players_temp.QB
    rb = players_temp.RB
    wr = players_temp.WR
    te = players_temp.TE
    dst = players_temp.DST

    ### Objective Function: Maximize Fantasy League Points
    length = len(players_temp)
    m.setObjective((gp.quicksum(points[x] * player[x] for x in range(length))), GRB.MAXIMIZE)

    ### Constraint 1: Positions
    # 1 QB
    m.addConstr(1 == gp.quicksum(player[x]*qb[x] for x in range(length)))

    # 2 RB (and FB) & maybe flex
    m.addConstr(2 <= sum(player[x]*rb[x] for x in range(length)))
    m.addConstr(3 >= sum(player[x]*rb[x] for x in range(length)))

    # 3 WR & maybe flex
    m.addConstr(3 <= gp.quicksum(player[x]*wr[x] for x in range(length)))
    m.addConstr(4 >= gp.quicksum(player[x]*wr[x] for x in range(length)))

    # 1 TE & maybe flex
    m.addConstr(1 <= gp.quicksum(player[x]*te[x] for x in range(length)))
    m.addConstr(2 >= gp.quicksum(player[x]*te[x] for x in range(length)))

    # 1 DST (Defense)
    m.addConstr(1 == gp.quicksum(player[x]*dst[x] for x in range(length)))

    # 1 FLEX: add an extra RB, WR or TE (total 9 players)
    m.addConstr(9 == gp.quicksum(player[x]*qb[x] + player[x]*rb[x] + player[x]*wr[x] + 
    player[x]*te[x] + player[x]*dst[x] for x in range(length)))


    ### Constraint 2: Budget
    # Total team budget for each week cannot exceed $50,000
    m.addConstr(50000 >= gp.quicksum(player[x]*cost[x] for x in range(length)))

    status = m.optimize()

    # Output and analysis
    ObjectiveValue=m.objVal

    #The optimal solution consists of the following
    dict_week_i = []
    for v in m.getVars():
        if v.x == 1: 
            # extract number (aka key) from variable name
            one = v.VarName.index('[')
            two = v.VarName.index(']')
            temp = int(v.VarName[one+1:two])

            row_player = players_temp.loc[[0]]
           
            dict_player_v = {"FPRank": players_temp.FPRank[temp],
                             "SRank": players_temp.SRank[temp],
                             "Name": players_temp.Name[temp],
                             "Position": players_temp.Position[temp],
                             "Week": players_temp.Week[temp],
                             "Team": players_temp.Team[temp],
                             "Opponent": players_temp.Opponent[temp],
                             "Salary": players_temp.Salary[temp],
                             "FantasyPoints": players_temp.FantasyPoints[temp]}

            dict_week_i.append(dict_player_v)

    df_week_i = pd.DataFrame(dict_week_i)
    
    position_categories = ["QB", "RB", "WR", "TE", "FLEX", "DST"] 
    df_week_i["Position"] = pd.Categorical(df_week_i["Position"], categories = position_categories)
    df_week_i.sort_values(by = "Position")
    
    return df_week_i

In [17]:
### Run the optimization model on weeks 1-16
for i in range(1,17):
    df = optimize_week(i)
    dfs.append(df)

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10 rows, 436 columns and 1669 nonzeros
Model fingerprint: 0xb9992e6b
Variable types: 0 continuous, 436 integer (436 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+03]
  Objective range  [1e-01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+04]
Found heuristic solution: objective 107.3600000
Presolve removed 1 rows and 193 columns
Presolve time: 0.00s
Presolved: 9 rows, 243 columns, 957 nonzeros
Found heuristic solution: objective 340.5600000
Variable types: 0 continuous, 243 integer (237 binary)

Root relaxation: cutoff, 5 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0       340.56000  340.56000  0.00%     -    0

Thread count was 8 (of 8 available processors)

Solution count 3: 288.64 266.08 86.56 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.886400000000e+02, best bound 2.886400000000e+02, gap 0.0000%
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10 rows, 388 columns and 1483 nonzeros
Model fingerprint: 0x0496e9cd
Variable types: 0 continuous, 388 integer (388 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+03]
  Objective range  [1e-01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+04]
Found heuristic solution: objective 79.2600000
Presolve removed 0 rows and 151 columns
Presolve time: 0.00s
Presolved: 10 rows, 237 columns, 919 nonzeros
Found heuristic solution: objective 311.3600000
Variable types: 0 continuous, 237 integer (231 binary)

Root relaxation: objective 3.136975e+02, 8 iterations, 0.00 seconds (0.00 work units)

H    0     0                     279.1600000  282.17333  1.08%     -    0s
H    0     0                     279.5600000  282.17333  0.93%     -    0s
     0     0  281.78222    0    2  279.56000  281.78222  0.79%     -    0s
     0     0  281.78222    0    2  279.56000  281.78222  0.79%     -    0s
     0     0 infeasible    0       279.56000  279.56000  0.00%     -    0s

Cutting planes:
  Gomory: 1
  Cover: 2
  MIR: 1
  GUB cover: 1

Explored 1 nodes (28 simplex iterations) in 0.03 seconds (0.01 work units)
Thread count was 8 (of 8 available processors)

Solution count 7: 279.56 279.16 278.96 ... 102.96

Optimal solution found (tolerance 1.00e-04)
Best objective 2.795600000000e+02, best bound 2.795600000000e+02, gap 0.0000%
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10 rows, 437 columns and 1669 nonzeros
Model fingerprint: 0xc973b010
Variable types: 0 continuous, 43

In [19]:
### Ouputs selected lineups for each week
for i in range(16,len(dfs)):
    display(dfs[i])

,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,12.0,Lamar Jackson,QB,1,BAL,MIA,6000.0,36.56
1,1.0,3.0,Christian McCaffrey,RB,1,CAR,LAR,8800.0,45.90
2,2.0,19.0,Austin Ekeler,RB,1,LAC,IND,5500.0,39.40
3,4.0,1.0,DeAndre Hopkins,WR,1,HOU,NO,8200.0,34.10
4,1.0,32.0,Sammy Watkins,WR,1,KC,JAX,5000.0,49.80
5,2.0,42.0,DeSean Jackson,WR,1,PHI,WAS,4500.0,38.40
6,3.0,65.0,John Ross,WR,1,CIN,SEA,3900.0,37.80
7,1.0,5.0,Evan Engram,TE,1,NYG,DAL,4800.0,31.60
8,1.0,27.0,San Francisco 49ers,DST,1,SF,TB,2200.0,27.00


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,1.0,Patrick Mahomes,QB,2,KC,LV,7500.0,35.62
1,1.0,6.0,Dalvin Cook,RB,2,MIN,GB,7200.0,31.10
2,2.0,21.0,Aaron Jones,RB,2,GB,MIN,5400.0,28.00
3,2.0,3.0,Odell Beckham Jr.,WR,2,CLE,NYJ,7800.0,31.10
4,3.0,8.0,Julio Jones,WR,2,ATL,PHI,7300.0,30.60
5,7.0,73.0,Nelson Agholor,WR,2,PHI,ATL,3700.0,27.70
6,1.0,80.0,Demarcus Robinson,WR,2,KC,LV,3500.0,38.20
7,1.0,8.0,Mark Andrews,TE,2,BAL,ARI,3800.0,28.20
8,1.0,3.0,New England Patriots,DST,2,NE,MIA,3700.0,37.00


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,6.0,Russell Wilson,QB,3,SEA,NO,6300.0,44.34
1,2.0,4.0,Alvin Kamara,RB,3,NO,SEA,8000.0,37.10
2,1.0,18.0,Mark Ingram II,RB,3,BAL,KC,5700.0,38.50
3,2.0,7.0,Keenan Allen,WR,3,LAC,HOU,7000.0,46.60
4,1.0,12.0,Mike Evans,WR,3,TB,NYG,6600.0,48.00
5,3.0,15.0,Cooper Kupp,WR,3,LAR,CLE,6400.0,36.10
6,5.0,74.0,Taylor Gabriel,WR,3,CHI,WAS,3600.0,32.20
7,1.0,8.0,Darren Waller,TE,3,LV,MIN,4100.0,30.10
8,2.0,31.0,New York Jets,DST,3,NYJ,NE,2100.0,16.00


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,13.0,Jameis Winston,QB,4,TB,LAR,5700.0,33.3
1,1.0,10.0,Nick Chubb,RB,4,CLE,BAL,6400.0,42.3
2,3.0,27.0,Jordan Howard,RB,4,PHI,GB,4600.0,32.5
3,6.0,27.0,Wayne Gallman,RB,4,NYG,WAS,4600.0,28.8
4,3.0,4.0,Davante Adams,WR,4,GB,PHI,7500.0,31.0
5,2.0,17.0,Robert Woods,WR,4,LAR,TB,6100.0,32.4
6,1.0,19.0,Chris Godwin,WR,4,TB,LAR,6000.0,44.2
7,1.0,7.0,Austin Hooper,TE,4,ATL,TEN,4300.0,25.0
8,1.0,2.0,New England Patriots,DST,4,NE,BUF,3700.0,25.0


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,2.0,12.0,Matt Ryan,QB,5,ATL,HOU,5900.0,35.9
1,2.0,1.0,Christian McCaffrey,RB,5,CAR,JAX,8700.0,50.7
2,1.0,17.0,Aaron Jones,RB,5,GB,DAL,5900.0,52.2
3,3.0,7.0,Amari Cooper,WR,5,DAL,GB,6800.0,42.6
4,2.0,10.0,Michael Thomas,WR,5,NO,TB,6600.0,44.2
5,4.0,34.0,DJ Chark Jr.,WR,5,JAX,CAR,5000.0,39.4
6,1.0,47.0,William Fuller V,WR,5,HOU,ATL,4500.0,56.7
7,1.0,31.0,Gerald Everett,TE,5,LAR,SEA,2900.0,23.6
8,1.0,3.0,Philadelphia Eagles,DST,5,PHI,NYJ,3700.0,35.0


...

,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,2.0,22.0,Ryan Tannehill,QB,12,TEN,JAX,5300.0,33.36
1,1.0,7.0,Leonard Fournette,RB,12,JAX,TEN,7300.0,36.90
2,6.0,41.0,Jonathan Williams,RB,12,IND,HOU,4200.0,24.10
3,1.0,6.0,Chris Godwin,WR,12,TB,ATL,7200.0,40.40
4,3.0,17.0,DJ Moore,WR,12,CAR,NO,6400.0,34.40
5,2.0,20.0,Jarvis Landry,WR,12,CLE,MIA,6300.0,39.80
6,7.0,44.0,A.J. Brown,WR,12,TEN,JAX,4700.0,26.50
7,2.0,3.0,Zach Ertz,TE,12,PHI,SEA,6000.0,27.10
8,3.0,19.0,Seattle Seahawks,DST,12,SEA,PHI,2600.0,17.00


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,26.0,Ryan Fitzpatrick,QB,13,MIA,PHI,5400.0,30.8
1,1.0,27.0,James White,RB,13,NE,HOU,5200.0,37.7
2,2.0,40.0,Derrius Guice,RB,13,WAS,CAR,4500.0,30.7
3,4.0,19.0,Kenny Golladay,WR,13,DET,CHI,6100.0,28.8
4,1.0,23.0,DeVante Parker,WR,13,MIA,PHI,5700.0,37.9
5,2.0,27.0,Robert Woods,WR,13,LAR,ARI,5500.0,33.2
6,3.0,40.0,Alshon Jeffery,WR,13,PHI,MIA,5100.0,31.7
7,1.0,48.0,Tyler Higbee,TE,13,LAR,ARI,2500.0,26.7
8,1.0,16.0,Tampa Bay Buccaneers,DST,13,TB,JAX,2900.0,23.0


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,18.0,Drew Brees,QB,14,NO,SF,5900.0,43.06
1,2.0,9.0,Aaron Jones,RB,14,GB,WAS,6700.0,34.20
2,1.0,13.0,Austin Ekeler,RB,14,LAC,JAX,6300.0,37.30
3,1.0,30.0,Emmanuel Sanders,WR,14,SF,NO,5800.0,37.10
4,2.0,36.0,A.J. Brown,WR,14,TEN,LV,5300.0,36.60
5,6.0,40.0,Robby Anderson,WR,14,NYJ,MIA,5100.0,28.10
6,3.0,45.0,Darius Slayton,WR,14,NYG,PHI,4900.0,35.40
7,1.0,2.0,Zach Ertz,TE,14,PHI,NYG,6100.0,30.10
8,1.0,4.0,Pittsburgh Steelers,DST,14,PIT,ARI,3500.0,18.00


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,1.0,Lamar Jackson,QB,15,BAL,NYJ,7500.0,37.08
1,4.0,5.0,Ezekiel Elliott,RB,15,DAL,LAR,7800.0,34.00
2,2.0,21.0,Miles Sanders,RB,15,PHI,WAS,5900.0,38.20
3,1.0,29.0,Kenyan Drake,RB,15,ARI,CLE,5000.0,42.60
4,1.0,9.0,Julio Jones,WR,15,ATL,SF,7000.0,41.40
5,4.0,38.0,Anthony Miller,WR,15,CHI,GB,5100.0,29.80
6,2.0,49.0,Breshad Perriman,WR,15,TB,DET,4500.0,37.60
7,3.0,13.0,Tyler Higbee,TE,15,LAR,DAL,3900.0,26.10
8,1.0,11.0,Minnesota Vikings,DST,15,MIN,LAC,3100.0,27.00


,FPRank,SRank,Name,Position,Week,Team,Opponent,Salary,FantasyPoints
0,1.0,13.0,Daniel Jones,QB,16,NYG,WAS,5900.0,38.28
1,1.0,3.0,Saquon Barkley,RB,16,NYG,WAS,8300.0,46.90
2,2.0,13.0,Kenyan Drake,RB,16,ARI,SEA,6300.0,36.40
3,4.0,18.0,Devonta Freeman,RB,16,ATL,JAX,6000.0,33.70
4,1.0,30.0,Tyler Boyd,WR,16,CIN,MIA,5800.0,36.80
5,6.0,62.0,Steven Sims,WR,16,WAS,NYG,4000.0,25.30
6,4.0,74.0,Hunter Renfrow,WR,16,LV,LAC,3600.0,26.70
7,1.0,5.0,Mark Andrews,TE,16,BAL,CLE,5900.0,27.30
8,1.0,7.0,Indianapolis Colts,DST,16,IND,CAR,3300.0,30.00


### Summary Statistics

In [22]:
dict_dfs = []

for df in dfs:
    
    df = df.sort_values(by='Salary', ascending=False)
    
    QB = df[df.Position == 'QB']
    RB = df[df.Position == 'RB']
    WR = df[df.Position == 'WR']
    TE = df[df.Position == 'TE']
    DST = df[df.Position == 'DST']
    
    dict_df = {
    "TotalFantasyPoints": df.FantasyPoints.sum(),
    "SalaryUsed": df.Salary.sum(),

    "SameTeamQB&RB": 1 if QB.Team.to_list()[0] in RB.Team.to_list() else 0,
    "SameTeamQB&WR": 1 if QB.Team.to_list()[0] in WR.Team.to_list() else 0,
    "SameTeamQB&TE": 1 if QB.Team.to_list()[0] in TE.Team.to_list() else 0,

    "SameTeamRB&WR": 1 if any(item in RB.Team.to_list() for item in WR.Team.to_list()) == True else 0,
    "SameTeamRB&TE": 1 if any(item in RB.Team.to_list() for item in TE.Team.to_list()) == True else 0,
    "SameTeamWR&TE": 1 if any(item in WR.Team.to_list() for item in TE.Team.to_list()) == True else 0,

    "DST=QBOpponent": 1 if DST.Team.to_list()[0] in QB.Opponent.to_list() else 0,
    "DST=RBOpponent": 1 if DST.Team.to_list()[0] in RB.Opponent.to_list() else 0,
    "DST=WROpponent": 1 if DST.Team.to_list()[0] in WR.Opponent.to_list() else 0,
    "DST=TEOpponent": 1 if DST.Team.to_list()[0] in TE.Opponent.to_list() else 0,

    "FlexRB": 1 if len(RB) > 2 else 0,
    "FlexWR": 1 if len(WR) > 3 else 0,
    "FlexTE": 1 if len(TE) > 1 else 0,
        
    "AvgQBSRank":stats.mean(QB.SRank.to_list()),
    "AvgRBSRank":np.round(stats.mean(RB.SRank.to_list()),2),
    "AvgWRSRank":np.round(stats.mean(WR.SRank.to_list()),2),
    "AvgTESRank":np.round(stats.mean(TE.SRank.to_list()),2),
    "AvgDSTSRank":stats.mean(DST.SRank.to_list()),
        
    "MinRBSRank":min(RB.SRank.to_list()),
    "MaxRBSRank":max(RB.SRank.to_list()),
        
    "MinWRSRank":min(WR.SRank.to_list()),
    "MaxWRSRank":max(WR.SRank.to_list()),
    
    "MinTESRank":min(TE.SRank.to_list()),
    "MaxTESRank":max(TE.SRank.to_list()),

    "Salary%QB": QB.Salary.sum()/50000,
    "Salary%RB": RB.Salary.sum()/50000 if len(RB) == 2 else RB.drop(RB.tail(1).index).Salary.sum()/50000,
    "Salary%WR": WR.Salary.sum()/50000 if len(WR) == 3 else WR.drop(WR.tail(1).index).Salary.sum()/50000,
    "Salary%TE": TE.Salary.sum()/50000 if len(TE) == 1 else TE.drop(TE.tail(1).index).Salary.sum()/50000,
    "Salary%DST": DST.Salary.sum()/50000,

    "FantasyPoints%QB": QB.FantasyPoints.sum()/df.FantasyPoints.sum(),
    "FantasyPoints%RB": RB.FantasyPoints.sum()/df.FantasyPoints.sum() if len(RB) <= 2 else RB.drop(RB.tail(1).index).FantasyPoints.sum()/df.FantasyPoints.sum(),
    "FantasyPoints%WR": WR.FantasyPoints.sum()/df.FantasyPoints.sum() if len(WR) <= 3 else WR.drop(WR.tail(1).index).FantasyPoints.sum()/df.FantasyPoints.sum(),
    "FantasyPoints%TE": TE.FantasyPoints.sum()/df.FantasyPoints.sum() if len(TE) <= 1 else TE.drop(TE.tail(1).index).FantasyPoints.sum()/df.FantasyPoints.sum(),
    "FantasyPoints%DST": DST.FantasyPoints.sum()/df.FantasyPoints.sum()}
    
    dict_dfs.append(dict_df)
    
summary = pd.DataFrame(dict_dfs)

summary.insert(0, "Week", list(range(1,17))+list(range(1,17)))

summary

,Week,TotalFantasyPoints,SalaryUsed,SameTeamQB&RB,SameTeamQB&WR,SameTeamQB&TE,SameTeamRB&WR,SameTeamRB&TE,SameTeamWR&TE,DST=QBOpponent,DST=RBOpponent,DST=WROpponent,DST=TEOpponent,FlexRB,FlexWR,FlexTE,AvgQBSRank,AvgRBSRank,AvgWRSRank,AvgTESRank,AvgDSTSRank,MinRBSRank,MaxRBSRank,MinWRSRank,MaxWRSRank,MinTESRank,MaxTESRank,Salary%QB,Salary%RB,Salary%WR,Salary%TE,Salary%DST,FantasyPoints%QB,FantasyPoints%RB,FantasyPoints%WR,FantasyPoints%TE,FantasyPoints%DST
0,1,288.78,49100.0,0,0,0,0,0,0,0,0,0,0,0,1,0,3.0,36.50,20.75,19.0,20.0,10.0,63.0,4.0,49.0,19.0,19.0,0.140,0.216,0.402,0.082,0.048,0.120438,0.218852,0.399958,0.093843,0.058868
1,2,293.50,50000.0,0,0,0,0,0,0,0,0,0,0,0,0,1,4.0,7.00,12.67,35.0,22.0,5.0,9.0,6.0,24.0,17.0,53.0,0.136,0.298,0.384,0.080,0.050,0.149233,0.296422,0.319250,0.102215,0.051107
2,3,314.50,49700.0,0,0,0,0,0,1,0,0,0,0,0,1,0,2.0,29.00,24.50,27.0,1.0,2.0,56.0,11.0,56.0,27.0,27.0,0.148,0.238,0.374,0.068,0.082,0.136725,0.262321,0.335453,0.076312,0.082671
3,4,320.18,48800.0,0,1,0,0,0,0,0,0,0,0,0,0,1,4.0,9.50,14.67,12.5,0.0,3.0,16.0,7.0,20.0,1.0,24.0,0.144,0.268,0.370,0.126,0.000,0.132051,0.239553,0.325754,0.134612,0.062465
4,5,289.20,45000.0,0,0,1,0,0,0,0,0,0,0,0,1,0,3.0,14.00,65.50,2.0,4.0,10.0,18.0,58.0,81.0,2.0,2.0,0.154,0.242,0.238,0.128,0.078,0.116528,0.200207,0.378976,0.096127,0.089903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,12,279.56,50000.0,0,1,0,0,0,0,0,0,0,1,0,1,0,22.0,24.00,21.75,3.0,19.0,7.0,41.0,6.0,44.0,3.0,3.0,0.106,0.230,0.398,0.120,0.052,0.119330,0.218200,0.409930,0.096938,0.060810
28,13,280.50,42900.0,0,1,0,0,0,1,0,0,0,0,0,1,0,26.0,33.50,27.25,48.0,16.0,27.0,40.0,19.0,40.0,48.0,48.0,0.108,0.194,0.346,0.050,0.058,0.109804,0.243850,0.356150,0.095187,0.081996
29,14,299.86,49600.0,0,0,0,0,0,0,0,0,0,0,0,1,0,18.0,11.00,37.75,2.0,4.0,9.0,13.0,30.0,45.0,2.0,2.0,0.118,0.260,0.324,0.122,0.070,0.143600,0.238445,0.339492,0.100380,0.060028
30,15,313.78,49800.0,0,0,0,0,0,0,0,0,0,0,1,0,0,1.0,18.33,32.00,13.0,11.0,5.0,29.0,9.0,49.0,13.0,13.0,0.150,0.274,0.332,0.078,0.062,0.118172,0.230098,0.346740,0.083179,0.086048


In [23]:
summary.iloc[:,1:3].describe()

,TotalFantasyPoints,SalaryUsed
count,32.000000,32.000000
mean,298.807500,48940.625000
std,26.531611,1854.525307
min,247.540000,42600.000000
25%,285.200000,49050.000000
50%,293.750000,49600.000000
75%,313.180000,49825.000000
max,380.300000,50000.000000


In [24]:
summary.iloc[:,3:16].sum()

SameTeamQB&RB      2
SameTeamQB&WR     12
SameTeamQB&TE      3
SameTeamRB&WR      3
SameTeamRB&TE      1
SameTeamWR&TE      3
DST=QBOpponent     0
DST=RBOpponent     1
DST=WROpponent     0
DST=TEOpponent     2
FlexRB            10
FlexWR            20
FlexTE             2
dtype: int64

In [25]:
summary.iloc[:,16:27].describe()

,AvgQBSRank,AvgRBSRank,AvgWRSRank,AvgTESRank,AvgDSTSRank,MinRBSRank,MaxRBSRank,MinWRSRank,MaxWRSRank,MinTESRank,MaxTESRank
count,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,8.781250,17.785938,28.841250,14.390625,13.468750,7.375000,27.437500,10.656250,52.062500,13.468750,15.312500
std,6.307877,9.764792,13.890213,13.715270,8.739914,7.618949,14.641826,13.362464,22.545921,13.394713,14.982651
min,1.000000,1.500000,9.670000,0.000000,0.000000,0.000000,2.000000,1.000000,13.000000,0.000000,0.000000
25%,4.000000,11.000000,19.000000,2.750000,4.000000,3.000000,17.750000,3.000000,39.500000,2.000000,2.750000
50%,7.000000,13.750000,27.375000,8.000000,13.500000,5.000000,23.000000,6.000000,49.000000,7.500000,8.000000
75%,13.000000,22.875000,35.125000,22.500000,20.000000,9.000000,40.250000,11.500000,71.750000,20.250000,24.750000
max,26.000000,38.000000,65.500000,48.000000,31.000000,31.000000,63.000000,58.000000,93.000000,48.000000,53.000000


In [26]:
summary.iloc[:,27:].describe()

,Salary%QB,Salary%RB,Salary%WR,Salary%TE,Salary%DST,FantasyPoints%QB,FantasyPoints%RB,FantasyPoints%WR,FantasyPoints%TE,FantasyPoints%DST
count,32.000000,32.000000,32.000000,32.000000,32.0000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,0.131375,0.250187,0.362875,0.088437,0.0575,0.126059,0.236701,0.358417,0.093344,0.076667
std,0.014710,0.046718,0.050601,0.033765,0.0160,0.014918,0.034297,0.035209,0.022030,0.018369
min,0.106000,0.106000,0.238000,0.000000,0.0000,0.094399,0.143123,0.292670,0.052517,0.047926
25%,0.118000,0.227500,0.336000,0.067500,0.0495,0.117322,0.215159,0.334146,0.082034,0.061581
50%,0.129000,0.256000,0.372000,0.082000,0.0560,0.124881,0.238999,0.354814,0.094515,0.079836
75%,0.142500,0.280000,0.398500,0.120500,0.0670,0.137706,0.264907,0.384477,0.099477,0.086913
max,0.162000,0.334000,0.464000,0.146000,0.0860,0.151230,0.296422,0.416570,0.159394,0.128687


In [27]:
dict_dfs = []

for df in dfs:
    
    df = df.sort_values(by='Salary', ascending=False)
    
    QB = df[df.Position == 'QB']
    RB = df[df.Position == 'RB']
    WR = df[df.Position == 'WR']
    TE = df[df.Position == 'TE']
    DST = df[df.Position == 'DST']
    
    if len(WR) == 4:
    
        dict_df = {
        "Week": QB.Week.to_list()[0],
         
        "QB_SRank": QB.SRank.to_list()[0],
        "QB_Salary": QB.Salary.to_list()[0],
        "QB_%Salary": QB.Salary.to_list()[0]/50000,
        
            
        "RB1_SRank": RB.SRank.to_list()[0],
        "RB1_Salary": RB.Salary.to_list()[0],
        "RB1_%Salary": RB.Salary.to_list()[0]/50000,
        "RB2_SRank": RB.SRank.to_list()[1],
        "RB2_Salary": RB.Salary.to_list()[1],
        "RB2_%Salary": RB.Salary.to_list()[1]/50000,
            
        "WR1_SRank": WR.SRank.to_list()[0],
        "WR1_Salary": WR.Salary.to_list()[0],
        "WR1_%Salary": WR.Salary.to_list()[0]/50000,
        "WR2_SRank": WR.SRank.to_list()[1],
        "WR2_Salary": WR.Salary.to_list()[1],
        "WR2_%Salary": WR.Salary.to_list()[1]/50000,
        "WR3_SRank": WR.SRank.to_list()[2],
        "WR3_Salary": WR.Salary.to_list()[2],
        "WR3_%Salary": WR.Salary.to_list()[2]/50000,
        "WR4_SRank": WR.SRank.to_list()[3],
        "WR4_Salary": WR.Salary.to_list()[3],
        "WR4_%Salary": WR.Salary.to_list()[3]/50000,
            
        "TE_SRank": TE.SRank.to_list()[0],
        "TE_Salary": TE.Salary.to_list()[0],
        "TE_%Salary": TE.Salary.to_list()[0]/50000,
                   
        "DST_SRank": DST.SRank.to_list()[0],
        "DST_Salary": DST.Salary.to_list()[0],
        "DST_%Salary": DST.Salary.to_list()[0]/50000,}
            
        dict_dfs.append(dict_df)
        
    else:
        continue
    
summary = pd.DataFrame(dict_dfs)

#SalaryFLEX = 1-(summary['Salary%QB']+summary['Salary%RB']+summary['Salary%WR']+summary['Salary%TE']+summary['Salary%DST'])
#summary.insert(18, "Salary%FLEX", SalaryFLEX)

#FantasyPointsFLEX = 1-(summary['FantasyPoints%QB']+summary['FantasyPoints%RB']+summary['FantasyPoints%WR']+summary['FantasyPoints%TE']+summary['FantasyPoints%DST'])
#summary.insert(25, "FantasyPoints%FLEX", FantasyPointsFLEX)

summary

,Week,QB_SRank,QB_Salary,QB_%Salary,RB1_SRank,RB1_Salary,RB1_%Salary,RB2_SRank,RB2_Salary,RB2_%Salary,WR1_SRank,WR1_Salary,WR1_%Salary,WR2_SRank,WR2_Salary,WR2_%Salary,WR3_SRank,WR3_Salary,WR3_%Salary,WR4_SRank,WR4_Salary,WR4_%Salary,TE_SRank,TE_Salary,TE_%Salary,DST_SRank,DST_Salary,DST_%Salary
0,1,3.0,7000.0,0.140,10.0,6800.0,0.136,63.0,4000.0,0.080,4.0,7300.0,0.146,9.0,6700.0,0.134,21.0,6100.0,0.122,49.0,4700.0,0.094,19.0,4100.0,0.082,20.0,2400.0,0.048
1,3,2.0,7400.0,0.148,2.0,7900.0,0.158,56.0,4000.0,0.080,11.0,6400.0,0.128,14.0,6200.0,0.124,17.0,6100.0,0.122,56.0,4200.0,0.084,27.0,3400.0,0.068,1.0,4100.0,0.082
2,5,3.0,7700.0,0.154,10.0,6400.0,0.128,18.0,5700.0,0.114,58.0,4100.0,0.082,58.0,4100.0,0.082,65.0,3700.0,0.074,81.0,3000.0,0.060,2.0,6400.0,0.128,4.0,3900.0,0.078
3,7,11.0,6400.0,0.128,9.0,6200.0,0.124,42.0,4000.0,0.080,2.0,7900.0,0.158,11.0,6600.0,0.132,14.0,6300.0,0.126,28.0,5400.0,0.108,37.0,2500.0,0.050,2.0,4300.0,0.086
4,8,1.0,8100.0,0.162,3.0,7500.0,0.150,42.0,4000.0,0.080,2.0,7500.0,0.150,32.0,5100.0,0.102,37.0,4700.0,0.094,49.0,4000.0,0.080,2.0,6600.0,0.132,22.0,2400.0,0.048
5,9,5.0,7000.0,0.140,1.0,8500.0,0.170,2.0,8200.0,0.164,48.0,4400.0,0.088,49.0,4300.0,0.086,61.0,3800.0,0.076,82.0,3000.0,0.060,1.0,7200.0,0.144,13.0,2900.0,0.058
6,11,8.0,6500.0,0.130,31.0,5100.0,0.102,43.0,4300.0,0.086,6.0,7400.0,0.148,14.0,6300.0,0.126,19.0,6000.0,0.120,71.0,3500.0,0.070,1.0,7300.0,0.146,10.0,3000.0,0.060
7,12,4.0,7400.0,0.148,3.0,7900.0,0.158,14.0,6000.0,0.120,4.0,7800.0,0.156,12.0,6400.0,0.128,35.0,5200.0,0.104,91.0,3000.0,0.060,21.0,3400.0,0.068,28.0,2200.0,0.044
8,13,15.0,5800.0,0.116,22.0,5500.0,0.110,54.0,4000.0,0.080,2.0,9000.0,0.180,12.0,6900.0,0.138,34.0,5100.0,0.102,38.0,4900.0,0.098,2.0,6100.0,0.122,22.0,2400.0,0.048
9,16,7.0,7300.0,0.146,25.0,5300.0,0.106,0.0,0.0,0.000,1.0,9100.0,0.182,5.0,8000.0,0.160,15.0,6100.0,0.122,46.0,4100.0,0.082,0.0,0.0,0.000,18.0,2700.0,0.054


In [28]:
df = summary.loc[:,['QB_SRank','RB1_SRank','RB2_SRank','WR1_SRank','WR2_SRank','WR3_SRank','WR4_SRank','TE_SRank','DST_SRank']]
df.describe()


,QB_SRank,RB1_SRank,RB2_SRank,WR1_SRank,WR2_SRank,WR3_SRank,WR4_SRank,TE_SRank,DST_SRank
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,9.000000,10.050000,29.150000,11.600000,19.400000,32.300000,55.150000,13.700000,14.950000
std,7.319764,9.605234,18.454139,15.908951,14.597765,17.100631,19.015991,14.400658,9.439419
min,1.000000,1.000000,0.000000,1.000000,5.000000,14.000000,24.000000,0.000000,1.000000
25%,3.000000,3.000000,16.250000,2.000000,9.750000,19.750000,43.500000,2.000000,4.000000
50%,7.000000,6.500000,21.500000,5.500000,13.000000,28.000000,49.000000,7.500000,17.000000
75%,12.750000,12.750000,42.250000,11.750000,25.250000,37.750000,71.750000,22.500000,20.500000
max,26.000000,31.000000,63.000000,58.000000,58.000000,73.000000,91.000000,48.000000,31.000000


In [29]:
df = summary.loc[:,['QB_Salary','RB1_Salary','RB2_Salary','WR1_Salary','WR2_Salary','WR3_Salary','WR4_Salary','TE_Salary','DST_Salary']]
df.describe()

,QB_Salary,RB1_Salary,RB2_Salary,WR1_Salary,WR2_Salary,WR3_Salary,WR4_Salary,TE_Salary,DST_Salary
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,6615.000000,7095.000000,4870.000000,7070.000000,6205.000000,5315.000000,4195.000000,4405.000000,2930.000000
std,816.458978,1464.123593,1609.543904,1283.539514,1012.799663,880.953431,791.716988,1898.884714,686.792698
min,5300.000000,5000.000000,0.000000,4100.000000,4100.000000,3700.000000,3000.000000,0.000000,2100.000000
25%,5900.000000,6025.000000,4000.000000,6400.000000,5600.000000,4925.000000,3575.000000,3200.000000,2400.000000
50%,6550.000000,7250.000000,4900.000000,7350.000000,6400.000000,5350.000000,4150.000000,4100.000000,2650.000000
75%,7325.000000,7925.000000,5750.000000,7800.000000,6750.000000,6100.000000,4750.000000,6100.000000,3550.000000
max,8100.000000,10500.000000,8200.000000,9100.000000,8000.000000,6400.000000,5700.000000,7300.000000,4300.000000


In [30]:
df = summary.loc[:,['QB_%Salary','RB1_%Salary','RB2_%Salary','WR1_%Salary','WR2_%Salary','WR3_%Salary','WR4_%Salary','TE_%Salary','DST_%Salary']]
df.describe()

,QB_%Salary,RB1_%Salary,RB2_%Salary,WR1_%Salary,WR2_%Salary,WR3_%Salary,WR4_%Salary,TE_%Salary,DST_%Salary
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.132300,0.141900,0.097400,0.141400,0.124100,0.106300,0.083900,0.088100,0.058600
std,0.016329,0.029282,0.032191,0.025671,0.020256,0.017619,0.015834,0.037978,0.013736
min,0.106000,0.100000,0.000000,0.082000,0.082000,0.074000,0.060000,0.000000,0.042000
25%,0.118000,0.120500,0.080000,0.128000,0.112000,0.098500,0.071500,0.064000,0.048000
50%,0.131000,0.145000,0.098000,0.147000,0.128000,0.107000,0.083000,0.082000,0.053000
75%,0.146500,0.158500,0.115000,0.156000,0.135000,0.122000,0.095000,0.122000,0.071000
max,0.162000,0.210000,0.164000,0.182000,0.160000,0.128000,0.114000,0.146000,0.086000
